**How to run dopo - an example**

Imports

In [1]:
import dopo
from dopo import*

In [2]:
import brightway2 as bw
import bw2data as bd

Set a brightway project, set up brightway, define an ecoinvent database and premise database.

In [3]:
bd.projects.set_current("premise-validation-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


Define the yaml filter files by adding them to a dictionary. Each dictionary key should represent a sector name and hold a file path and a 'yaml identifier'. This identifier corresponds to the heading of a sector/ group of activities in the filter file.

In [4]:
files_dict={}
files_dict['Cement']={'yaml': r'C:\Users\fried\premise_validation\dev\yamls\cement_small.yaml', 
                      'yaml identifier': 'Cement'}
files_dict['Steel']= {'yaml':r'C:\Users\fried\premise_validation\dev\yamls\steel_small.yaml',
                           'yaml identifier': 'Steel'}
#files_dict['Fuel']={'yaml': r'C:\Users\fried\premise_validation\dopo\yamls\fuels_small.yaml',
                   # 'yaml identifier': 'Fuels' }
files_dict

{'Cement': {'yaml': 'C:\\Users\\fried\\premise_validation\\dev\\yamls\\cement_small.yaml',
  'yaml identifier': 'Cement'},
 'Steel': {'yaml': 'C:\\Users\\fried\\premise_validation\\dev\\yamls\\steel_small.yaml',
  'yaml identifier': 'Steel'}}

With the created files dictionary and a database as input, the function process_yaml_files filters 
the activities in the database for each sector and stores them in a dictionary of dictionaries. 
The structure of the dictionary e.g. for steel: premise_dict['Steel']['activities'].
Here two dictionaries are created for both premise and ecoinvent database for later comparison.

In [5]:
premise_dict = process_yaml_files(files_dict, ei39SSP2)
ecoinvent_dict = process_yaml_files(files_dict, ei39)


Processing Cement with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Cement:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '3c16b45db40210cd97de6574b2f47aaf')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'a3c2064d83411f7963af550c04c869a1')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'df49e8f525497f2fbd56bcdc80ff0cde')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '36a53c174f34e672bc15b7e55563685e')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '86841f8c7ee2668f244d3b8e34f41932')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'fcb666edf2a01467e555eeff5b4a5bbb')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'f8b84f45f50d3bd7ff4feaabdb493f6a')
Processing Steel with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Steel:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '2baa0deb3adc89dfe8cb89d5e078ba8d')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '1dffacc9e0ca08fb55c6b780d7e677dc

For calculcating LCA scores, methods need to be selected. This can be done using the class MethodFinder.
If the keywords are known for selecting a unique method they can be entered directly into the function otherwise some previous filtering should be considered to find the right method. For each method find_and_create_method is called and calling finder.get_all_methods() stores them together in one method dictionary.

In [6]:
finder=dopo.methods.MethodFinder()

finder.find_and_create_method(criteria=['IPCC', '2013', 'GWP100'], exclude=['no LT'])
finder.find_and_create_method(criteria=['EN15804','Cumulative', 'non-renewable' ])
finder.find_and_create_method(criteria=['land occupation','selected'])
finder.find_and_create_method(criteria=['EN15804','fresh water'])
method_dict=finder.get_all_methods()
method_dict

{'method_1': {'object': Brightway2 Method: IPCC 2013: climate change: global warming potential (GWP100),
  'method name': ('IPCC 2013',
   'climate change',
   'global warming potential (GWP100)'),
  'short name': 'global warming potential (GWP100)',
  'unit': 'kg CO2-Eq'},
 'method_2': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: Cumulative Energy Demand - non-renewable energy resources,
  'method name': ('EN15804',
   'inventory indicators ISO21930',
   'Cumulative Energy Demand - non-renewable energy resources'),
  'short name': 'Cumulative Energy Demand - non-renewable energy resources',
  'unit': 'megajoule'},
 'method_3': {'object': Brightway2 Method: selected LCI results: resource: land occupation,
  'method name': ('selected LCI results', 'resource', 'land occupation'),
  'short name': 'land occupation',
  'unit': 'square meter-year'},
 'method_4': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: use of net fresh water,
  'method na

With the created dictionaries LCA score tables and excel plots can be generated using the subsequent function. An excel filepath or filename (inlc. 'xlsx' ending) needs to be defined in addition. The resulting plots are: (a) dot/scatter plots: Plots are created by sector on one worksheet. The dot plots plot the total lca score for each activitiy in that sector and by method ranking them from highest to lowest. The score and unit are given on the y-axis. On the x-axis the rank number for the activity is given. The activity to the rank can be found in the respective sector table. (b) stacked bar plots: by sector and method stacked bar plots are created which disaggreagted the total lca score by input and visualizes the input contributions. A cutoff value for the input contribution can be defined (default: 0.01). On the x-axis the score and unit are indicated. On the y-axis the rank of the activity is given. The charts are stored on the same sheet as the dot plots.

In [7]:
sector_lca_scores_plots(premise_dict, method_dict, '0509_v1.xlsx', cutoff=0.01)

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '
key in method_dfs
Cement_global_warming_potential_(gwp100)
key in method_dfs
Cement_cumulative_energy_demand_-_non-renewable_energy_resources
key in method_dfs
Cement_land_occupation
key in method_dfs
Cement_use_of_net_fresh_water
key in method_dfs
Steel_global_warming_potential_(gwp100)
key in method_dfs
Steel_cumulative_energy_demand_-_non-renewable_energy_resources
key in method_dfs
Steel_land_occupation
key in method_dfs
Steel_use_of_net_fresh_water
last row occupied in excel

The function database_comparison_plots caclulates total LCA scores for all activities in the defined dictionaries for ecoinvent and premise. It then plots the relative change of the scores for each activity by sector and method. These plots are stored on the same sector worksheet as related dot and stacked bar plots. Therefore, the previous function prints out the last row the stacked bar chart occupied in all chart worksheets. This number defines the current_row argument of the next function. 

In [9]:
database_comparison_plots(ecoinvent_dict, premise_dict, method_dict,'0509_v1.xlsx',current_row=93)

df_ei
                      activity  \
0  cement production, Portland   
1  cement production, Portland   
2  cement production, Portland   
3  cement production, Portland   
4  cement production, Portland   
5  cement production, Portland   
6  cement production, Portland   

                                        activity key reference product  \
0  (ecoinvent 3.9.1 cutoff, 36a53c174f34e672bc15b...  cement, Portland   
1  (ecoinvent 3.9.1 cutoff, 86841f8c7ee2668f244d3...  cement, Portland   
2  (ecoinvent 3.9.1 cutoff, fcb666edf2a01467e555e...  cement, Portland   
3  (ecoinvent 3.9.1 cutoff, f8b84f45f50d3bd7ff4fe...  cement, Portland   
4  (ecoinvent 3.9.1 cutoff, a3c2064d83411f7963af5...  cement, Portland   
5  (ecoinvent 3.9.1 cutoff, 3c16b45db40210cd97de6...  cement, Portland   
6  (ecoinvent 3.9.1 cutoff, df49e8f525497f2fbd56b...  cement, Portland   

  location                             method method unit     total  \
0       US  global warming potential (GWP100)   kg CO2-Eq